In [1]:
import os

In [2]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification\\research'

In [3]:
os.chdir('c:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification')

In [4]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    training_data:str
    trained_model_path: Path
    updated_base_model_path: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    params_is_augmentation: bool

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'cnnClassifier'

In [7]:
class ConfigurationManager:
  def __init__(
      self,
      config_filepath:Path=CONFIG_FILE_PATH,
      params_filepath:Path=PARAMS_FILE_PATH
  ):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])

  def get_training_config(self) -> TrainingConfig:
    training = self.config.training
    prepare_base_model = self.config.prepare_base_model
    params = self.params
    training_data = os.path.join(self.config.data_ingestion.unzip_dir, "C:/Users/quamr/OneDrive/Desktop/project/kidneyDiseaseClassification/artifacts/data_ingestion/content/drive/MyDrive/CT-kidney-data")
    create_directories([training.root_dir])

    training_config = TrainingConfig(
      root_dir = Path(training.root_dir),
      training_data = training_data,
      trained_model_path = Path(training.trained_model_path),
      updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
      params_batch_size=params.BATCH_SIZE,
      params_image_size=params.IMAGE_SIZE,
      params_epochs=params.EPOCHS,
      params_is_augmentation=params.AUGMENTATION  
    )
    return training_config

NameError: name 'CONFIG_FILE_PATH' is not defined

In [101]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [102]:
class Training:
  def __init__(self, config:TrainingConfig):
    self.config = config
  
  def get_base_model(self):
    self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

  def train_valid_generator(self):
    datagenerator_kwargs = dict(
        rescale=1./255,
        validation_split=0.20
    )
    dataflow_kwargs = dict(
        target_size=self.config.params_image_size[:-1],
        batch_size=self.config.params_batch_size,
        interpolation='bilinear'
    )
    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

    self.valid_generator = valid_datagenerator.flow_from_directory(
        directory=self.config.training_data,
        subset="validation",
        shuffle=False,
        **dataflow_kwargs
    )

    if self.config.params_is_augmentation:
      train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
         rotation_range=40,
         horizontal_flip=True,
         width_shift_range=0.2,
         height_shift_range=0.2,
         shear_range=0.2,
         zoom_range=0.2,
         **datagenerator_kwargs
      )
    else:
      train_datagenerator = valid_datagenerator
    
    self.train_generator = train_datagenerator.flow_from_directory(
      directory = self.config.training_data,
      subset="training",
      shuffle=True,
      **dataflow_kwargs
    )
  @staticmethod
  def save_model(path:Path, model:tf.keras.Model):
   model.save(path)


  def train(self):
   self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
   self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size


   self.model.fit(
        self.train_generator,
        validation_data=self.valid_generator,
        epochs=self.config.params_epochs,
        steps_per_epoch=self.steps_per_epoch,
        validation_steps=self.validation_steps
    )


   self.save_model(
    path = self.config.trained_model_path,
    model = self.model
  )

In [103]:
try:
  config = ConfigurationManager()
  training_config = config.get_training_config()
  training = Training(config=training_config)
  training.get_base_model()
  training.train_valid_generator()
  training.train()
except Exception as e:
  raise e

[2025-12-01 16:46:06,121: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-01 16:46:06,141: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-01 16:46:06,143: INFO: common: created directory at: artifacts]
[2025-12-01 16:46:06,145: INFO: common: created directory at: artifacts/training]
Found 2489 images belonging to 4 classes.
Found 9969 images belonging to 4 classes.
Epoch 1/3
623/623 [==============================] - 1620s 3s/step - loss: 13.9535 - accuracy: 0.4127 - val_loss: 6.5982 - val_accuracy: 0.3226
Epoch 2/3
623/623 [==============================] - 4625s 7s/step - loss: 7.0246 - accuracy: 0.5479 - val_loss: 19.2059 - val_accuracy: 0.4472
Epoch 3/3
623/623 [==============================] - 1559s 3s/step - loss: 5.5808 - accuracy: 0.6022 - val_loss: 6.3736 - val_accuracy: 0.4085


C:\Users\quamr\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
